In [1]:
import numpy as np
from operator import itemgetter

In [2]:
rows = open("07.txt", "r").read().split("\n")

In [3]:
import re

patt = re.compile("^Step ([A-Z]) must be finished before step ([A-Z]) can begin.$")

In [4]:
deps_list = [patt.match(x).groups() for x in rows]
deps_list[:10]

[('D', 'T'),
 ('T', 'K'),
 ('O', 'S'),
 ('Y', 'G'),
 ('F', 'Q'),
 ('Q', 'C'),
 ('R', 'X'),
 ('E', 'X'),
 ('V', 'U'),
 ('U', 'H')]

In [5]:
from collections import defaultdict

keys = {x for dep in deps_list for x in dep}
parents = {k: [] for k in keys}

for k, v in deps_list:
    parents[k].append(v)

parents

{'X': ['S', 'G', 'H', 'K', 'C'],
 'B': ['J', 'I', 'N'],
 'C': ['N', 'J', 'I'],
 'K': ['W', 'I'],
 'Y': ['G', 'R', 'M', 'J', 'S', 'W', 'N', 'V'],
 'U': ['H', 'M', 'X', 'J', 'I'],
 'G': ['I', 'Z', 'B'],
 'S': ['J', 'C', 'N', 'G'],
 'E': ['X', 'B', 'U', 'K', 'Z'],
 'F': ['Q', 'Z', 'K', 'A', 'P', 'M', 'C', 'I'],
 'O': ['S', 'L', 'P', 'M'],
 'Z': ['I', 'B', 'N', 'C'],
 'Q': ['C', 'N', 'W', 'A', 'U', 'P'],
 'I': ['N'],
 'L': ['C', 'A', 'W', 'J'],
 'M': ['J', 'C', 'N', 'G', 'B'],
 'A': ['N', 'J', 'K', 'I', 'M'],
 'V': ['U', 'C', 'B', 'I', 'G', 'W'],
 'P': ['B', 'I', 'G', 'K'],
 'H': ['Z', 'N', 'J'],
 'D': ['T', 'C'],
 'N': [],
 'W': ['C', 'B', 'J', 'S'],
 'T': ['K', 'E', 'L'],
 'J': ['N', 'I'],
 'R': ['X', 'J']}

In [6]:
def getdeps():
    deps = defaultdict(list)

    for k, v in deps_list:
        deps[v].append(k)

    return dict(deps)


getdeps()

{'T': ['D'],
 'K': ['T', 'F', 'A', 'X', 'E', 'P'],
 'S': ['O', 'X', 'Y', 'W'],
 'G': ['Y', 'X', 'V', 'P', 'M', 'S'],
 'Q': ['F'],
 'C': ['Q', 'L', 'W', 'V', 'S', 'M', 'F', 'Z', 'X', 'D'],
 'X': ['R', 'E', 'U'],
 'U': ['V', 'E', 'Q'],
 'H': ['U', 'X'],
 'B': ['P', 'Z', 'W', 'V', 'E', 'G', 'M'],
 'N': ['A', 'C', 'J', 'I', 'Q', 'H', 'B', 'S', 'M', 'Z', 'Y'],
 'W': ['K', 'Q', 'L', 'V', 'Y'],
 'Z': ['H', 'F', 'G', 'E'],
 'J': ['M', 'S', 'B', 'A', 'R', 'W', 'U', 'Y', 'L', 'C', 'H'],
 'I': ['G', 'Z', 'J', 'B', 'V', 'P', 'A', 'K', 'U', 'C', 'F'],
 'L': ['O', 'T'],
 'R': ['Y'],
 'M': ['U', 'Y', 'A', 'F', 'O'],
 'E': ['T'],
 'A': ['L', 'Q', 'F'],
 'P': ['O', 'F', 'Q'],
 'V': ['Y']}

In [7]:
deps = getdeps()
free_jobs = list(keys - set(deps.keys()))
res = []
while len(free_jobs):
    ch = min(free_jobs)
    free_jobs.remove(ch)
    res.append(ch)
    pars = parents[ch]
    for par in pars:
        deps[par].remove(ch)
        if not len(deps[par]):
            free_jobs.append(par)
assert 26 == len(res)
"".join(res)

'DFOQPTELAYRVUMXHKWSGZBCJIN'

Part II

In [8]:
def jobtime(char):
    return ord(char) - 64 + 60


jobtime("J")

70

In [9]:
busy_until = np.zeros(5)
deps = getdeps()
free_jobs = list(keys - set(deps.keys()))
free_jobs = [(x, 0) for x in free_jobs]
res = []
time = 0
while len(free_jobs):
    # print(time)
    # at least one worker is available
    # these can start now.
    available_jobs = [x for x in free_jobs if x[1] <= time]

    if not len(available_jobs):
        # no jobs available. wait unitl the next one is:
        time = min(x[1] for x in free_jobs)
        continue

    # print(free_jobs)
    # print(busy_until)
    # take the first alphabetically:
    el = min(available_jobs, key=itemgetter(0))
    free_jobs.remove(el)
    ch, ready = el
    # print(ch)
    res.append(ch)
    job_ready = time + jobtime(ch)
    busy_until[0] = job_ready
    busy_until.sort()
    # wait until the next elve is available:
    time = busy_until[0]
    pars = parents[ch]
    for par in pars:
        deps[par].remove(ch)
        if not len(deps[par]):
            free_jobs.append((par, job_ready))

In [10]:
max(busy_until)

1036.0